In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import fastai
fastai.__version__

In [ ]:
from fastai import *
from fastai_audio import *
from fastai.vision import models

In [ ]:
DATA = Path('data')
AUDIO = DATA/'freesound/audio_44KHz'
CSV = DATA/'freesound/audio_44KHz/train.csv'

In [ ]:
df = pd.read_csv(CSV)
df['fname'] = df['fname'].str[:-4]
df.head(3)

In [ ]:
n_fft = 2048
n_hop = 512
n_mels = 128
sr = 44100
top_db = 60.0

bs = 16

In [ ]:
mel_spec_tfm = Compose([
    Spectrogram(n_fft=n_fft, n_hop=n_hop),
    FrequencyToMel(n_mels=n_mels, n_fft=n_fft, sr=sr, f_min=0., f_max=None),
    SpectrumToDb(ref='max', top_db=top_db, normalized=True)
])

def inputs_tfm(inputs):
    x, y = inputs
    # 1d time domain signal to 2d log melspectrogram (frequency domain)
    x = mel_spec_tfm(x)
    # add channel dimension
    x.unsqueeze_(1)
    return x, y

tfms = [inputs_tfm]

In [ ]:
# subset of df
df = df.iloc[:32]
data = (AudioItemList
            .from_df(df, path=AUDIO, folder='train', suffix='.wav')
            .random_split_by_pct()
            .label_from_df()
            .databunch(bs=bs, tfms=tfms, equal_lengths=False))

In [ ]:
xs, ys = data.train_dl.one_batch()
xs.shape, ys.shape, xs.min(), xs.max()

In [ ]:
model_name = 'freesound_44khz_2048_512_128_60db_v1'

learn = create_cnn(data, models.resnet18, metrics=[error_rate, mapk])

In [ ]:
learn.fit_one_cycle(8)
learn.save(model_name + '-stage-1')